##**1. Download Dataset**

Download the ucmerced landuse dataset with 21 classes where each class has 100 images. The 21 classes are:

* agricultural
* airplane
* baseballdiamond
* beach
* buildings
* chaparral
* denseresidential
* forest
* freeway
* golf course
* harbor
* intersection
* mediumresidential
* mobilehomepark
* overpass
* parking lot
* river
* runway
* sparseresidential
* storage tanks
* tennis court

Where each image has a size of 256x256 RGB pixels.




##**2. Mountain Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**3. Import Important Library**

In [2]:
import os
from skimage.io import imread, imsave
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import cv2
import matplotlib.pyplot as plt

##**4.  Extract Dataset**

In [3]:
try:
  os.mkdir('data')
except:
  print("Error, there is already a 'data' folder")

In [6]:
data_set = os.path.join('/content/drive/MyDrive/ON_JOB_TRAINING/Images/')

In [7]:
labels = os.listdir('/content/drive/MyDrive/ON_JOB_TRAINING/Images/')
labels.sort()
labels

['agricultural',
 'airplane',
 'baseballdiamond',
 'beach',
 'buildings',
 'chaparral',
 'denseresidential',
 'forest',
 'freeway',
 'golfcourse',
 'harbor',
 'intersection',
 'mediumresidential',
 'mobilehomepark',
 'overpass',
 'parkinglot',
 'river',
 'runway',
 'sparseresidential',
 'storagetanks',
 'tenniscourt']

##**5. Divide Dataset into Training, Validation, Testing**

In [8]:
def complement (a,b):
    f2=[]
    for x in a:
        x=os.path.splitext(x)[0]+'.tif'
        if x not in b:
            f2.append(x)
    return (f2)

In [9]:
train_data_size=80
validate_data_size=10
test_data_size=10
for j in labels:
    path=os.path.join('/content/drive/MyDrive/ON_JOB_TRAINING/Images/',j)
    path1_train=os.path.join('data','train',j)
    path1_validate=os.path.join('data','validation',j)
    path1_test=os.path.join('data','test',j)
    
    try:
      shutil.rmtree(path1_train)
      os.makedirs(path1_train)
      shutil.rmtree(path1_validate)
      os.makedirs(path1_validate)
      shutil.rmtree(path1_test) 
      os.makedirs(path1_test) 
    except:
      os.makedirs(path1_train) 
      os.makedirs(path1_validate) 
      os.makedirs(path1_test) 
    
    
    files1= os.listdir(path)
    files1=files1[1:len(files1)]
    files1= np.random.permutation(files1)
    for i in range(0,train_data_size):
        file=os.path.join(path,files1[i])
        img1=imread(file)
        n=os.path.splitext(file)
        n=n[0].split('/')
        n1=os.path.join(path1_train, n[-1] +'.png')
        imsave(n1,img1)

    print(path1_train, len(os.listdir(path1_train)))
    validate_data0 = complement(files1, os.listdir(path1_train))
    
    validate_data = np.random.permutation(validate_data0)
    for i in range( 0, validate_data_size):
        file=os.path.join(path,validate_data[i])
        img1=imread(file)
        n=os.path.splitext(file)
        n=n[0].split('/')
        n1=os.path.join(path1_validate, n[-1] +'.png')
        imsave(n1,img1)
    print(path1_validate, len(os.listdir(path1_validate))) 
    
    test_data0 = complement(validate_data0, os.listdir(path1_validate))
    test_data = np.random.permutation(test_data0)
    for i in range(0, test_data_size):
        file=os.path.join(path,test_data[i])
        img1=imread(file)
        n=os.path.splitext(file)
        n=n[0].split('/')
        n1=os.path.join(path1_test, n[-1] +'.png')
        imsave(n1,img1)
        
    print(path1_test, len(os.listdir(path1_test)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: data/train/agricultural/agricultural55.png is a low contrast image


data/train/agricultural 80
data/validation/agricultural 10
data/test/agricultural 10
data/train/airplane 80
data/validation/airplane 10
data/test/airplane 10
data/train/baseballdiamond 80
data/validation/baseballdiamond 10
data/test/baseballdiamond 10
data/train/beach 80
data/validation/beach 10
data/test/beach 10
data/train/buildings 80
data/validation/buildings 10
data/test/buildings 10
data/train/chaparral 80
data/validation/chaparral 10
data/test/chaparral 10
data/train/denseresidential 80
data/validation/denseresidential 10
data/test/denseresidential 10
data/train/forest 80
data/validation/forest 10
data/test/forest 10
data/train/freeway 80
data/validation/freeway 10
data/test/freeway 10
data/train/golfcourse 80
data/validation/golfcourse 10
data/test/golfcourse 10
data/train/harbor 80
data/validation/harbor 10
data/test/harbor 10
data/train/intersection 80
data/validation/intersection 10
data/test/intersection 10
data/train/mediumresidential 80
data/validation/mediumresidential 1

##**6. Preprocess Image for Training**

In [10]:
from keras.preprocessing.image import ImageDataGenerator

base_directory=path=os.path.join('/content/UCMerced_LandUse/Images')
train_dir=os.path.join('data','train')
validation_dir=os.path.join('data','validation')
test_dir=os.path.join('data','test')

train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical'
    )

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.)
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical'
    )

test_datagen = ImageDataGenerator(rescale=1.0 / 255.)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=1,
        color_mode='rgb',
        class_mode='categorical'
    )
X_test = []
y_test = []
for i in range (len(test_generator)):
  x,y = test_generator[i]
  X_test.append(x)
  y_test.append(y)
X_test = np.array(X_test).reshape(210,150,150,3)
y_test = np.array(y_test).reshape(210,21)
y_test_class = [np.argmax(y) for y in y_test]
class_label = {v: k for k, v in test_generator.class_indices.items()}
class_label

Found 1680 images belonging to 21 classes.
Found 210 images belonging to 21 classes.
Found 210 images belonging to 21 classes.


{0: 'agricultural',
 1: 'airplane',
 2: 'baseballdiamond',
 3: 'beach',
 4: 'buildings',
 5: 'chaparral',
 6: 'denseresidential',
 7: 'forest',
 8: 'freeway',
 9: 'golfcourse',
 10: 'harbor',
 11: 'intersection',
 12: 'mediumresidential',
 13: 'mobilehomepark',
 14: 'overpass',
 15: 'parkinglot',
 16: 'river',
 17: 'runway',
 18: 'sparseresidential',
 19: 'storagetanks',
 20: 'tenniscourt'}

##**7. Build CNN Model**
###**7.1 Train CNN Model**

In [ ]:
model = Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,150,3),
               data_format='channels_last'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,)

model.save(f"/content/drive/Shareddrives/Data/Model Land Use/model_CNN Acc:{history.history['accuracy'][-1]}.h5")

Epoch 1/50
53/53 [==============================] - 55s 1s/step - loss: 3.0646 - accuracy: 0.0774 - val_loss: 2.7758 - val_accuracy: 0.2000
Epoch 2/50
53/53 [==============================] - 53s 994ms/step - loss: 2.7866 - accuracy: 0.1524 - val_loss: 2.4716 - val_accuracy: 0.2333
Epoch 3/50
53/53 [==============================] - 53s 992ms/step - loss: 2.6154 - accuracy: 0.1815 - val_loss: 2.3251 - val_accuracy: 0.2571
Epoch 4/50
53/53 [==============================] - 53s 991ms/step - loss: 2.4884 - accuracy: 0.2131 - val_loss: 2.1371 - val_accuracy: 0.3143
Epoch 5/50
53/53 [==============================] - 55s 1s/step - loss: 2.4298 - accuracy: 0.2286 - val_loss: 2.2266 - val_accuracy: 0.3143
Epoch 6/50
53/53 [==============================] - 53s 993ms/step - loss: 2.3106 - accuracy: 0.2655 - val_loss: 2.0118 - val_accuracy: 0.3429
Epoch 7/50
53/53 [==============================] - 53s 993ms/step - loss: 2.2153 - accuracy: 0.2821 - val_loss: 1.7958 - val_accuracy: 0.4286
Epoch

###**7.2 Plot Model Accuracy and Loss**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(acc, label='Training acc')
plt.plot(val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

###**7.3 Evaluate Model Performance**

In [ ]:
model.evaluate(test_generator)

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
print(classification_report(y_test_class, y_pred_class))

##**7.4 Plot Confusion Matrix**
Plot the confusion matrix to see where the model successfully predicts and fails to predict



In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    plt.figure(figsize=(20,20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes,rotation=90)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f'
    thresh = cm.max()/2.0
    
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        
        plt.text(j,i, format(cm[i,j], fmt),
                horizontalalignment = "center",
                color = "white" if cm[i,j] > thresh else "black")
        pass
    
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.grid(None);
    pass

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
cnf_mat = confusion_matrix(y_test_class, y_pred_class)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=labels)
plt.grid(None)
plt.show();

##**8. Transfer Learning with VGG16**
###**8.1 Load PreTrained Model**

In [ ]:
conv_base = tf.keras.applications.VGG16(weights='imagenet',include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = False
conv_base.summary()

###**8.2 Build Model with Pretrained Model as Feature Extractor**

In [ ]:
for layer in conv_base.layers:
    layer.trainable = False

top_model = conv_base.output
top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
top_model = tf.keras.layers.Dense(4096, activation='relu')(top_model)
top_model = tf.keras.layers.Dense(1072, activation='relu')(top_model)
top_model = tf.keras.layers.Dropout(0.5)(top_model)
output_layer = tf.keras.layers.Dense(21, activation='softmax')(top_model)
    
# Group the convolutional base and new fully-connected layers into a Model object.
model = Model(inputs=conv_base.input, outputs=output_layer)

# Compiles the model for training.
model.compile(optimizer=Adam(learning_rate=2e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
history = model.fit(train_generator,
                    epochs=20,
                    batch_size=32,
                    validation_data=validation_generator)
model.save(f"/content/drive/Shareddrives/Data/Model Land Use/model_VGG Acc:{history.history['accuracy'][-1]}.h5")

###**8.3 Plot Model Accuracy and Loss**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(acc, label='Training acc')
plt.plot(val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

###**8.4  Evaluate Model Performance**

In [ ]:
model.evaluate(test_generator)

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
print(classification_report(y_test_class, y_pred_class))

###**8.5 Plot Confusion Matrix**

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
cnf_mat = confusion_matrix(y_test_class, y_pred_class)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=labels)
plt.grid(None)
plt.show();

##**9. Transfer Learning with MobileNet**
###**9.1 Build Model with Pretrained Model as Feature Extractor**

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
mobile = MobileNet(weights="imagenet",include_top=False,input_shape=(150,150,3))

for layer in mobile.layers:
    layer.trainable = False

top_model = mobile.output
top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
top_model = tf.keras.layers.Dense(4096, activation='relu')(top_model)
top_model = tf.keras.layers.Dense(1072, activation='relu')(top_model)
top_model = tf.keras.layers.Dropout(0.5)(top_model)
output_layer = tf.keras.layers.Dense(21, activation='softmax')(top_model)
    
# Group the convolutional base and new fully-connected layers into a Model object.
model = Model(inputs=mobile.input, outputs=output_layer)

# Compiles the model for training.
model.compile(optimizer=Adam(learning_rate=2e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
history = model.fit(train_generator,
                    epochs=20,
                    batch_size=32,
                    validation_data=validation_generator)
model.save(f"/content/drive/Shareddrives/Data/Model Land Use/model_MobileNet Acc:{history.history['accuracy'][-1]}.h5")

###**9.2 Plot Model Accuracy and Loss**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(acc, label='Training acc')
plt.plot(val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

###**9.3 Evaluate Model Performance**

In [ ]:
model.evaluate(test_generator)

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
print(classification_report(y_test_class, y_pred_class))

###**9.4 Plot Confusion Matrix**

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
cnf_mat = confusion_matrix(y_test_class, y_pred_class)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=labels)
plt.grid(None)
plt.show();

##**10. Transfer Learning with InceptionV3**
###**10.1 Build Model with Pretrained Model as Feature Extractor**

In [ ]:
from keras.applications.inception_v3 import InceptionV3
inc = InceptionV3(input_shape=(150, 150, 3),
                                    include_top=False)

for layer in inc.layers:
    layer.trainable = False

top_model = inc.output
top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
top_model = tf.keras.layers.Dense(4096, activation='relu')(top_model)
top_model = tf.keras.layers.Dense(1072, activation='relu')(top_model)
top_model = tf.keras.layers.Dropout(0.5)(top_model)
output_layer = tf.keras.layers.Dense(21, activation='softmax')(top_model)
    
# Group the convolutional base and new fully-connected layers into a Model object.
model = Model(inputs=inc.input, outputs=output_layer)

# Compiles the model for training.
model.compile(optimizer=Adam(learning_rate=2e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
history = model.fit(train_generator,
                    epochs=20,
                    batch_size=32,
                    validation_data=validation_generator)

model.save(f"/content/drive/Shareddrives/Data/Model Land Use/model_Inception Acc:{history.history['accuracy'][-1]}.h5")

###**10.2 Plot Model Accuracy and Loss**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(acc, label='Training acc')
plt.plot(val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

###**10.3 Evaluate Model Performance**

In [ ]:
model.evaluate(test_generator)

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
print(classification_report(y_test_class, y_pred_class))

###**10.4 Plot Confusion Matrix**

In [ ]:
y_pred = model.predict(X_test)
y_pred_class = [np.argmax(y) for y in y_pred]
cnf_mat = confusion_matrix(y_test_class, y_pred_class)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=labels)
plt.grid(None)
plt.show();

##**11. Test with random image**
###**11.1 Load Model with Best Accuracy**

In [ ]:
import re
_path = "/content/drive/Shareddrives/Data/Model Land Use"
models = os.listdir(_path)
best_acc = 0
best_model = ""

regex = r"(\d+)(?:\.(\d{1,}))"
for _model in models:
  acc = re.search(regex,_model)
  cur_acc = float(acc.group(0))
  if cur_acc > best_acc:
    best_acc = cur_acc
    best_model = _model

model= tf.keras.models.load_model(os.path.join(_path, best_model))
print(f"Model used: {best_model}")

###**11.2 Upload Image to Test the Model**

<!-- from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  img = cv2.imread(fn)
  img = cv2.resize(img,(150,150))
  img = np.reshape(img,[1,150,150,3])
  img_arr = np.zeros((1,150,150,3))
  img_arr[0, :, :, :] = img / 255.
  y_pred = model.predict(img_arr)
  plt.imshow(imread(fn))
  plt.axis('off')
  plt.show()
  print(f"Predicted as: {class_label[np.argmax(y_pred)]}") -->